# Import

In [17]:
import pandas as pd
import os
import torch
import tiktoken
from llm.previous_chapters import *

if torch.cuda.is_available():
    print("gpu available")
else:
    print("no gpu")

no gpu


In [3]:
root_dataset_path = os.path.join(os.getcwd(), "datasets", "spam_dataset")
dataset_path = os.path.join(root_dataset_path, "SMSSpamCollection")
assert os.path.exists(dataset_path), f"path to dataset not exists {dataset_path}"

In [4]:
df = pd.read_csv(dataset_path, sep="\t", header=None, names =['label', 'text'])
print(df.shape)
df.head()

(5572, 2)


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
#undersample for testing
def create_balanced_dataset(df: pd.DataFrame):
    num_spam = df[df["label"] == "spam"].shape[0]
    ham_subset = df[df["label"] == "ham"].sample(num_spam, random_state=123)
    #make # of ham == # spam
    balanced_df = pd.concat([ham_subset, df[df["label"] == 'spam']])
    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df['label'].value_counts())

label
ham     747
spam    747
Name: count, dtype: int64


In [6]:
def random_split(df, train_frac, valid_frac):
    #shuffle
    df = df.sample(frac = 1, random_state = 123).reset_index(drop=True)

    train_idx = int(train_frac * len(df))
    val_idx = train_idx + int(valid_frac * len(df))
    #the rest is test

    train_df = df[:train_idx]
    val_df = df[train_idx:val_idx]
    test_df = df[val_idx:]

    return train_df, val_df, test_df

train_df, val_df, test_df = random_split(df, 0.7, 0.1)

print("train df shape = ", train_df.shape)
print("valid df shape = ", val_df.shape)
print("test df shape = ", test_df.shape)

train_df.to_csv(os.path.join(root_dataset_path, "sample_train.csv"), index = False)
val_df.to_csv(os.path.join(root_dataset_path, "sample_val.csv"), index = False)
test_df.to_csv(os.path.join(root_dataset_path, "sample_test.csv"), index = False)

train df shape =  (3900, 2)
valid df shape =  (557, 2)
test df shape =  (1115, 2)


In [7]:
#Map label
balanced_df['label'] = balanced_df['label'].map({'ham': 0, 'spam': 1})
balanced_df.head()

,label,text
4307,0,Awww dat is sweet! We can think of something t...
4138,0,Just got to &lt;#&gt;
4831,0,"The word ""Checkmate"" in chess comes from the P..."
4461,0,This is wishing you a great day. Moji told me ...
5440,0,Thank you. do you generally date the brothas?


# Pad tokens with longest token len

In [8]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
pad_id = tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})[0]
print(pad_id)

50256


# Build dataset and dataloader

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

In [10]:
class SpamDataset(Dataset):

    def __init__(self, csv, tokenizer, pad_token_id, max_len = 200):
        self.df = pd.read_csv(csv)
        self.df.drop(0, inplace=True)
        self.df['label'] = self.df['label'].map({'ham': 0, 'spam': 1})
        self.pad_token_id = pad_token_id
        self.max_len = max_len


        self.encoded_text = [self._pad_tokens(tokenizer.encode(text)) for text in self.df['text']]

    def _pad_tokens(self, tokens):
        tokens = tokens + [self.pad_token_id] * (self.max_len - len(tokens))
        return tokens
    
    def __getitem__(self, index):
        X = self.encoded_text[index]
        y = self.df.iloc[index]['label']
        
        return (torch.tensor(X).long(), torch.tensor(y).long())
    
    def __len__(self):
        return len(self.encoded_text)
    
    def get_dataloader(self, batch_size, num_workers, drop_last = False):
        return DataLoader(dataset = self, batch_size=batch_size, num_workers=num_workers, drop_last = drop_last)

In [11]:
train_dataset = SpamDataset(os.path.join(root_dataset_path, "sample_train.csv"), tokenizer, pad_id, max_len = 100)
train_dataset.df.head()
print(train_dataset[0])
print(len(train_dataset))


(tensor([ 5122,  1736,  1077,   415, 17442,   272,   338,  2802,  3804,  1497,
          938,  1755,    13, 12472,   329,   607,   290,  1641,    13, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]), tensor(0))
3899


In [12]:
val_dataset = SpamDataset(os.path.join(root_dataset_path, "sample_val.csv"), tokenizer, pad_id, max_len = 100)
print(len(val_dataset))
test_dataset = SpamDataset(os.path.join(root_dataset_path, "sample_test.csv"), tokenizer, pad_id, max_len = 100)
print(len(test_dataset))

556
1114


In [13]:
#Get dataloader
batch_size = 8
num_workers = 0

train_loader = train_dataset.get_dataloader(batch_size=batch_size, num_workers=num_workers, drop_last=True) #drop last unmatched element for training
val_loader = val_dataset.get_dataloader(batch_size=batch_size, num_workers=num_workers, drop_last=False)
test_loader = test_dataset.get_dataloader(batch_size=batch_size, num_workers=num_workers, drop_last=False)

print("train loader len = ", len(train_loader))
sample = next(iter(train_loader))
print(f"X shape = {sample[0].shape}, y shape = {sample[1].shape}")
print("val loader len = ", len(val_loader))
print("test loader len = ", len(test_loader))

train loader len =  487
X shape = torch.Size([8, 100]), y shape = torch.Size([8])
val loader len =  70
test loader len =  140


In [14]:
print("train loader len = ", len(train_loader))
print("val loader len = ", len(val_loader))
print("test loader len = ", len(test_loader))

train loader len =  487
val loader len =  70
test loader len =  140


# Loading pretrained model. 
Code in Chapt 5, load from book's code, my GPT code is messed up
because my GPTCode is slightly different than theirs

In [34]:
BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}
CHOOSE_MODEL = "gpt2-small (124M)"

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

In [18]:
gpt = torch.load(os.path.join("output", 'gpt.torch'))

In [28]:
text = "we are the world" 
tokens = text_to_token_ids(text, tokenizer)

output_tokens = generate(gpt, tokens, max_new_tokens=10, context_size=BASE_CONFIG["context_length"], temperature=2.0, top_k=10)

decoded = token_ids_to_text(output_tokens, tokenizer)

print(decoded)

we are the world's first, best-known international music company,


In [31]:
# Sample instruction
text2 = ("Is the following spam, answer with yes or no", 
"'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'")

tokens = text_to_token_ids(text, tokenizer)

output_tokens = generate(gpt, tokens, max_new_tokens=10, context_size=BASE_CONFIG["context_length"], temperature=2.0, top_k=10)

decoded = token_ids_to_text(output_tokens, tokenizer)

print(decoded)

we are the world's only company that provides the world with all the


In [38]:
num_classes = 2
#Freeze model params for finetuning
for param in gpt.parameters():
    param.requires_grad = False

gpt.out_head = nn.Linear(
    BASE_CONFIG["emb_dim"],
    out_features=num_classes
)


#NOTE from the book: 
# training the output layer we just added is sufficient. 
# However, as I found in experiments, finetuning additional layers can 
# noticeably improve the predictive performance of the finetuned model.

for p in gpt.trf_blocks[-1].parameters():
    p.requires_grad = True
for p in gpt.final_norm.parameters():
    p.requires_grad = True

In [71]:
# test new model architecture
text = [
    "This definitely a spam email", 
    "this is not spam email"
    ]
ids = [text_to_token_ids(t, tokenizer).squeeze(0) for t in text]
ids = torch.vstack(ids)

print('input shape = ', ids.shape)

with torch.no_grad():
    preds = gpt(ids)

print('output shape = ', preds.shape)
# we want to use last output token [-1] to get optimize
# probas = torch.softmax(preds[:, -1, :], dim = -1)
# print(f"probas ({probas.shape})= ", probas)
logits = preds[:, -1, :]
label = torch.argmax(logits, dim = -1)

print('labels = ', label)

input shape =  torch.Size([2, 5])
output shape =  torch.Size([2, 5, 2])
labels =  tensor([1, 1])


# Evaluation function

In [100]:
def calc_accuracy_loader(dataloader:DataLoader, model:GPTModel, device, num_batches = None):
    model.eval()

    correct_pred, num_examples = 0,0

    if num_batches != None: 
        num_batches = min(num_batches, len(dataloader))
    else:
        num_batches = len(dataloader)
    
    for i, (X, y) in enumerate(dataloader):
        if i >= num_batches: break

        X, y = X.to(device), y.to(device)

        with torch.no_grad():
            logits = model(X)[:,-1,:]
        
        preds = torch.argmax(logits, dim = -1)

        num_examples += preds.shape[0]

        correct_pred += (preds == y).sum().item()
    
    return correct_pred / num_examples

def calc_loss_loader(dataloader:DataLoader, model:GPTModel, device, num_batches = None):
    model.eval()

    total_loss, num_examples = 0,0

    if num_batches != None: 
        num_batches = min(num_batches, len(dataloader))
    else:
        num_batches = len(dataloader)
    
    for i, (X, y) in enumerate(dataloader):
        if i >= num_batches: break

        X, y = X.to(device), y.to(device)

        with torch.no_grad():
            logits = model(X)[:,-1,:]

        num_examples += preds.shape[0]
        total_loss  += calc_loss_batch(X, y, model, device)
    
    return total_loss / num_examples

def calc_loss_batch(X: torch.tensor, y, model: torch.tensor, device: GPTModel):
    X, y = X.to(device), y.to(device)
    logits = model(X)[:, -1, :]
    loss = torch.nn.functional.cross_entropy(logits, y)
    return loss


def eval_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():

        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    
    model.train()
    return train_loss, val_loss

In [90]:
device = torch.device('device' if torch.cuda.is_available() else 'cpu')
print(device)
train_accuracy = calc_accuracy_loader(train_loader, gpt, device = device, num_batches= 10)
print("train acc = ", train_accuracy)

cpu
train acc =  0.8625


In [93]:
X, y = next(iter(train_loader))
train_loss = calc_loss_batch(X, y, gpt, device = device)
print("train loss = ", train_loss)

train loss =  tensor(0.5944, grad_fn=<NllLossBackward0>)


In [95]:
train_loss = calc_loss_loader(train_loader, gpt, device = device, num_batches= 10)
print("train loss = ", train_loss)

train loss =  tensor(0.2966, grad_fn=<DivBackward0>)


In [101]:
train_loss, val_loss = eval_model(gpt, train_loader,  val_loader, device, eval_iter=5)
print('train loss = ', train_loss)
print('val loss = ', val_loss)

train loss =  tensor(0.2911)
val loss =  tensor(0.3117)


# Training loop

In [110]:
from tqdm import tqdm

def train(
    model: GPTModel, train_loader: DataLoader, val_loader: DataLoader, 
    optimizer, device, tokenizer, num_epochs = 10, eval_freq = 1, eval_iter = 1):

    train_losses, val_losses, train_accs, val_accs = [],[],[],[]
    examples_seen, global_steps = 0,-1
    loop = tqdm(range(num_epochs))

    for e in loop:

        model.train()

        for X,y in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(X, y, model, device)
            loss.backward()
            optimizer.step()

            examples_seen += X.shape[0]

            global_steps += 1
        
            if global_steps % eval_freq == 0: 
                train_loss, val_loss = eval_model(model, train_loader, val_loader, device, eval_iter)

                train_losses.append(train_loss)
                val_losses.append(val_loss)
            
                print(f"epochs = {e}, global step ={global_steps}, train_loss={train_loss:.3f}, val_loss={val_loss:.3f}")
            
        train_acc = calc_accuracy_loader(train_loader, model, device,num_batches=eval_iter)
        val_acc = calc_accuracy_loader(val_loader, model, device,num_batches=eval_iter)

        loop.set_description(f"train acc = {train_acc:.3f}, val_acc = {val_acc:.3f}")

        train_accs.append(train_acc)
        val_accs.append(val_acc)
    
    return train_losses, val_losses, train_accs, val_accs, examples_seen


In [111]:
import time

start_time = time.time()
optimizer = torch.optim.AdamW(gpt.parameters(), lr = 5e-5, weight_decay=0.1)

num_epochs = 5

train(gpt, train_loader, val_loader, optimizer = optimizer, tokenizer=tokenizer, device = device, eval_iter = 10)


  0%|          | 0/10 [00:00<?, ?it/s]

epochs = 0, global step =0, train_loss=0.195, val_loss=0.197
epochs = 0, global step =1, train_loss=0.195, val_loss=0.198


  0%|          | 0/10 [00:30<?, ?it/s]


KeyboardInterrupt: 